<a href="https://colab.research.google.com/github/jmunsell1195/NLPproject/blob/main/NLPwithBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This program processes student essays that outline their approach to solving a problem. We use the BERT model from the simple transformers library to make predictions. BERT transforms raw text into contextualized word embeddings. The target variable is correctnesss on the problem.  

In [ ]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import textblob
from textblob import TextBlob

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Import data from google drive
downloaded = drive.CreateFile({'id':"1BZobq1AdzZNT8Z6h9PqR3EG8hy0LfyTm"})   # replace the id with id of file you want to access
downloaded.GetContentFile('172Q3P6.xls')   

#https://drive.google.com/file/d/1yfOgSHc3QdQixetyQFr2GGbGIPLBmX6Q/view?usp=sharing
downloaded = drive.CreateFile({'id':"1yfOgSHc3QdQixetyQFr2GGbGIPLBmX6Q"})   # replace the id with id of file you want to access
downloaded.GetContentFile('P172-Sp21-Quiz3_P6-LastAttempt_172ALL-Data.xlsx')  

train = pd.read_excel('172Q3P6.xls')
test = pd.read_excel('P172-Sp21-Quiz3_P6-LastAttempt_172ALL-Data.xlsx')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Load previously processed data (.pkl)

In [ ]:
# Load pickled data to avoid having to clean
# the data every time the program is run

import gc
import pickle
from google.colab import drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Training Set (.pkl)

trainPKLread = open('drive/My Drive/trainNLP.pkl','rb')
train = pickle.load(trainPKLread)
trainPKLread.close()

# Testing Set (.pkl)

testPKLread = open('drive/My Drive/testNLP.pkl','rb')
test = pickle.load(testPKLread)
testPKLread.close()

Mounted at /content/drive


Function and class to process data

In [ ]:
#Function clean takes a string (essay) as an argument.
#       -Removes Stop Words
#       -Removes Unnecessary chars
#       -Contextually unaware spelling correction

def clean(txt):
    # Set stopwords (english from NLTK library)
    stop = stopwords.words('english')
   
    # Remove punctuation and symbols (equations) which are essentially noise
    punc = ['0','1','2','3','4','5','6','7','8','9','!','@','#','$','%','^','&','*','(',')','~','`','[',']','{','}','|','\'','?','>','<',':',';',',','+','_','-','=']
   
    # Remove new line characters and split on space character
    txt = txt.replace('\n','').split(' ')
   
    # Append words only if they are not stop words (I,am,...)
    txt = [word for word in txt if word not in stop]
    text = []
   
    # For each non-stop word
    for word in txt:
   
      # Split words char-wise and append to list if not in punc
      chars = [char for char in word if char not in punc]
      word = ''.join(chars)

      # If not empty string
      if word != '':
        text.append(word.lower())
    text = ' '.join(text)

    # Context unaware spelling correction
    text = TextBlob(text)
    text = str(text.correct())
    return text


class dataPrep:
  """
  --------------------------dataPrep--------------------------------
  Class has two methods that are responsible for structuring data 
  in a format that makes BERT happy. Namely, a dx2 dimensional array 
  or dataframe with one column marked 'text' and another marked 'label'
  Also, the data came for two different sources and columns have 
                   different names and meanings


  ----Attributes----
  d1 ==> pandas dataframe with test data
  d2 ==> pandas dataframe with training data

  -----Methods-----
  set1Format ==> drops nuissance rows and renames columns. Applies clean function to
                 each essay

  set2Format ==> drops nuissance rows and renames columns. Applies clean function to 
                  each essay

  """
  
  def __init__(self,testdf,traindf):

    # Class constructed with train and test data set
    self.d1 = testdf
    self.d2 = traindf
  
  def set1Format(self):

    # drop un-needed columns 
    self.d1.drop([0,1,2,3,4],inplace=True)
    # rename columns in correct format
    self.d1[['PUID','label','text']] = self.d1[['Unnamed: 2','Unnamed: 8','Unnamed: 34']]
    # Keep only the necessary columns
    self.d1 = self.d1.loc[:,['PUID','label','text']]
    # drop any rows with missing values (label or essay)
    self.d1.dropna(how='any',inplace=True)
    # drop duplicates, keeping only the first attemp
    self.d1.drop_duplicates(subset='PUID',keep='first',inplace=True)

    # Now that we dropped duplicates, we are done the student ID (PUID)
    self.d1 = self.d1.loc[:,['text','label']]

    # Clean each essay
    self.d1['text'] = self.d1['text'].apply(lambda x: clean(x))
    
    # Return the data frame with cleaned text 
    return self.d1
  
  def set2Format(self):

    # rename columns in correct format
    self.d2[['User','label','text']] = self.d2[['Username','Awarded (OA)','A5R']]
    # Keep only necessary columns
    self.d2 = self.d2.loc[:,['User','label','text']]
    # Drop any remaining rows with missing values (label and essay)
    self.d2.dropna(how='any',inplace=True)
    # Drop duplicates keeping first
    self.d2.drop_duplicates(subset='User',keep='first',inplace=True)
    
    # Now that we've dropped duplicates, we don't need the Student ID (PUID)
    self.d2 = self.d2.loc[:,['text','label']]

    # Clean text
    self.d2['text'] = self.d2['text'].apply(lambda x: clean(x))

    #Return data frame with cleaned essays
    return self.d2

In [ ]:
#Check shapes of train/test

print(train.shape)
print(test.shape)

(1441, 2)
(1480, 2)


Code is commented because cleaned data was pickled and loaded previously

In [ ]:
# dataSet = dataPrep(test,train)

In [ ]:
# train = dataSet.set2Format()
# test = dataSet.set1Format()

Pickles the processed data sets

In [ ]:
# create and store train and test .pkl files 

# import gc
# import pickle
# from google.colab import drive

# trainNLPout = open('drive/My Drive/trainNLP.pkl','wb')
# pickle.dump(train, trainNLPout)
# trainNLPout.close()


# testNLPout = open('drive/My Drive/testNLP.pkl','wb')
# pickle.dump(test, testNLPout)
# testNLPout.close()


In [ ]:
!pip install simpletransformers

     |████████████████████████████████| 247 kB 8.3 MB/s 
     |████████████████████████████████| 6.8 MB 46.1 MB/s 
     |████████████████████████████████| 306 kB 67.5 MB/s 
     |████████████████████████████████| 9.2 MB 58.3 MB/s 
     |████████████████████████████████| 43 kB 2.5 MB/s 
     |████████████████████████████████| 1.2 MB 59.4 MB/s 
     |████████████████████████████████| 1.7 MB 65.1 MB/s 
     |████████████████████████████████| 3.4 MB 62.7 MB/s 
     |████████████████████████████████| 3.3 MB 62.6 MB/s 
     |████████████████████████████████| 61 kB 670 kB/s 
     |████████████████████████████████| 596 kB 55.7 MB/s 
     |████████████████████████████████| 895 kB 54.8 MB/s 
     |████████████████████████████████| 140 kB 76.1 MB/s 
     |████████████████████████████████| 97 kB 9.3 MB/s 
     |████████████████████████████████| 180 kB 67.8 MB/s 
     |████████████████████████████████| 63 kB 2.3 MB/s 
     |████████████████████████████████| 132 kB 73.2 MB/s 
     |█████████████████

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import wandb

In [ ]:
# Arguments (parameters) were adjusted using a random split of the data
# using wandb experiment visualization tool 

args= ClassificationArgs()
args.overwrite_output_dir = True

#train for 4 epochs
args.num_train_epochs = 4

#allow reprocessing input text
args.reprocess_input_data = True
args.use_multiprocessing = True

# use a split of data to stop parameter tuning 
#during training when perfomance stops increasing
args.evaluate_during_training = False
args.use_early_stopping = True

# batch size
args.train_batch_size= 75
args.polynomial_decay_schedule_lr_end = 1e-6
'''
args.wandb_project = "172 classification"

sweep_config = {
    "name": "172 classification",
    "method": "Bayes",
    "metric": {"name": "train_loss", "goal": "minimize"},
    "parameters": {
        "num_train_epochs": {"min": 1, "max": 40},
        "learning_rate": {"min": 0, "max": 4e-4},
    },
    "early_terminate": {"type": "hyperband", "min_iter": 6,},
}

sweep_id = wandb.sweep(sweep_config, project="172 classification")
'''

# Create a TransformerModel with modified attributes, binary labels, use cuda
model = ClassificationModel("bert", "bert-base-cased",num_labels=2,use_cuda=True,args=args)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Model validation using WandB

In [ ]:
train,val = train_test_split(train,test_size=0.3)

In [ ]:
# model.train_model(train)

In [ ]:
# model.evaluate(val,output_dir='output')

In [ ]:
model.train_model(train,eval_df=val)

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:586: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1008 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/14 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/14 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/14 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/14 [00:00<?, ?it/s]

(56, 0.5592690237930843)

In [ ]:
model.evaluate(test,output_dir='output')

/usr/local/lib/python3.7/dist-packages/simpletransformers/classification/classification_model.py:1427: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  "Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels."


  0%|          | 0/1480 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/185 [00:00<?, ?it/s]

({'auprc': 0.8122388842659741,
  'auroc': 0.8260339156319697,
  'eval_loss': 0.5025831067884291,
  'fn': 209,
  'fp': 105,
  'mcc': 0.5736402232016236,
  'tn': 696,
  'tp': 470},
 array([[-0.59130859,  0.32836914],
        [-0.92822266,  0.64794922],
        [-0.80615234,  0.58691406],
        ...,
        [ 0.59277344, -1.44335938],
        [ 0.54150391, -1.42675781],
        [-1.32324219,  1.07128906]]),
 [])

In [ ]:
preds = model.predict(test['text'].to_list())

  0%|          | 0/1480 [00:00<?, ?it/s]

  0%|          | 0/185 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
print(classification_report(np.array(test['label'],dtype=int),preds[0]))

              precision    recall  f1-score   support

           0       0.77      0.87      0.82       801
           1       0.82      0.69      0.75       679

    accuracy                           0.79      1480
   macro avg       0.79      0.78      0.78      1480
weighted avg       0.79      0.79      0.79      1480



In [ ]:
print(confusion_matrix(np.array(test['label'],dtype=int),preds[0]))

[[696 105]
 [209 470]]
